# Explore Hyperliquid Explorer Blocks

This notebook explores the structure and content of Hyperliquid's `explorer_blocks` data.

In [1]:
import json
import msgpack
import lz4.frame
from pathlib import Path
from collections import Counter, defaultdict
from datetime import datetime

SAMPLES_DIR = Path('../hyperliquid_samples/explorer_blocks')

## Load Sample Blocks

In [2]:
def load_blocks(filepath):
    """Load blocks from .rmp or .json file"""
    filepath = Path(filepath)
    
    if filepath.suffix == '.json':
        with open(filepath) as f:
            return json.load(f)
    elif filepath.suffix == '.rmp':
        with open(filepath, 'rb') as f:
            return msgpack.unpack(f, raw=False)
    elif filepath.suffix == '.lz4':
        with open(filepath, 'rb') as f:
            decompressed = lz4.frame.decompress(f.read())
            return msgpack.unpackb(decompressed, raw=False)
    else:
        raise ValueError(f'Unknown format: {filepath.suffix}')

In [3]:
# Load early blocks (Feb 2023)
blocks_early = load_blocks(SAMPLES_DIR / '1000.json')
print(f"Early blocks: {len(blocks_early)} blocks")
print(f"Range: {blocks_early[0]['header']['height']} - {blocks_early[-1]['header']['height']}")
print(f"Time: {blocks_early[0]['header']['block_time']}")

Early blocks: 100 blocks
Range: 901 - 1000
Time: 2023-02-26T17:41:39.942659


In [4]:
# Load mid-2023 blocks (Nov 2023)
blocks_mid = load_blocks(SAMPLES_DIR / '100000100.json')
print(f"Mid blocks: {len(blocks_mid)} blocks")
print(f"Range: {blocks_mid[0]['header']['height']} - {blocks_mid[-1]['header']['height']}")
print(f"Time: {blocks_mid[0]['header']['block_time']}")

Mid blocks: 100 blocks
Range: 100000001 - 100000100
Time: 2023-11-13T13:34:22.255707


In [5]:
# Load recent blocks (Nov 2025)
blocks_recent = load_blocks(SAMPLES_DIR / '811681900.json')
print(f"Recent blocks: {len(blocks_recent)} blocks")
print(f"Range: {blocks_recent[0]['header']['height']} - {blocks_recent[-1]['header']['height']}")
print(f"Time: {blocks_recent[0]['header']['block_time']}")

Recent blocks: 100 blocks
Range: 811681801 - 811681900
Time: 2025-11-28T22:44:28.368940020


## Analyze Action Types Over Time

In [6]:
def count_action_types(blocks):
    """Count action types in blocks"""
    action_types = Counter()
    for block in blocks:
        for tx in block.get('txs', []):
            for action in tx.get('actions', []):
                action_types[action.get('type', 'unknown')] += 1
    return action_types

print("=== Early (Feb 2023) ===")
for t, count in count_action_types(blocks_early).most_common():
    print(f"  {t}: {count}")

print("\n=== Mid (Nov 2023) ===")
for t, count in count_action_types(blocks_mid).most_common():
    print(f"  {t}: {count}")

print("\n=== Recent (Nov 2025) ===")
for t, count in count_action_types(blocks_recent).most_common():
    print(f"  {t}: {count}")

=== Early (Feb 2023) ===
  order: 100
  cancel: 25
  SetGlobalAction: 5
  CreditBridgeDepositAction: 1
  connect: 1

=== Mid (Nov 2023) ===
  order: 13608
  cancel: 13086
  SetGlobalAction: 8
  NetChildVaultPositionsAction: 3
  connect: 1

=== Recent (Nov 2025) ===
  order: 92577
  cancelByCloid: 14061
  cancel: 8336
  noop: 4267
  evmRawTx: 378
  batchModify: 315
  scheduleCancel: 265
  modify: 89
  updateLeverage: 13
  perpDeploy: 8
  SetGlobalAction: 3
  setReferrer: 3
  usdClassTransfer: 3
  subAccountTransfer: 2
  approveBuilderFee: 2
  NetChildVaultPositionsAction: 1
  createSubAccount: 1
  vaultTransfer: 1


## Block Structure

In [7]:
# Show block header structure
print("Block Header:")
print(json.dumps(blocks_recent[0]['header'], indent=2))

Block Header:
{
  "block_time": "2025-11-28T22:44:28.368940020",
  "height": 811681801,
  "hash": "0x95d0e496f2bb0e014eb64d23e2604c2d62446a70be747bd6de6f844f7fd0ec55",
  "proposer": "0x8c323b484d301933cd7a11c12bebd173f36933e4"
}


In [8]:
# Show sample transaction with order
for block in blocks_recent:
    for tx in block.get('txs', []):
        for action in tx.get('actions', []):
            if action.get('type') == 'order':
                print("Sample Order Transaction:")
                print(json.dumps(tx, indent=2, default=str))
                break
        else:
            continue
        break
    else:
        continue
    break

Sample Order Transaction:
{
  "actions": [
    {
      "type": "order",
      "orders": [
        {
          "a": 1,
          "b": true,
          "p": "3034.7",
          "s": "2.5966",
          "r": false,
          "t": {
            "limit": {
              "tif": "Alo"
            }
          },
          "c": "0x000000000000000000000000042ef3a0"
        }
      ],
      "grouping": "na"
    }
  ],
  "user": "0x847bd1ef4fede0fe191129894d63044843f4d473",
  "raw_tx_hash": null,
  "error": null
}


## Order Schema Reference

```python
{
    "type": "order",
    "orders": [{
        "a": 0,           # asset ID (0=BTC, 1=ETH, etc.)
        "b": true,        # true=buy, false=sell
        "p": "67500.0",   # price
        "s": "0.1",       # size
        "r": false,       # reduce only
        "t": {            # order type
            "limit": {
                "tif": "Alo"  # Alo=AddLiquidityOnly, Ioc=ImmediateOrCancel, Gtc=GoodTilCancel
            }
        },
        "c": "0x..."      # client order ID (optional)
    }],
    "grouping": "na"
}
```

## Extract Unique Addresses

In [9]:
def extract_addresses(blocks):
    """Extract unique addresses from blocks"""
    addresses = set()
    for block in blocks:
        for tx in block.get('txs', []):
            user = tx.get('user')
            if user:
                addresses.add(user)
    return addresses

addrs_early = extract_addresses(blocks_early)
addrs_mid = extract_addresses(blocks_mid)
addrs_recent = extract_addresses(blocks_recent)

print(f"Unique addresses in early blocks: {len(addrs_early)}")
print(f"Unique addresses in mid blocks: {len(addrs_mid)}")
print(f"Unique addresses in recent blocks: {len(addrs_recent)}")

Unique addresses in early blocks: 2
Unique addresses in mid blocks: 23
Unique addresses in recent blocks: 742


In [10]:
# Show sample addresses
print("Sample addresses from recent blocks:")
for addr in list(addrs_recent)[:10]:
    print(f"  {addr}")

Sample addresses from recent blocks:
  0xd72cfd2424e26ba32e46b7b2aff95eff34247a71
  0x8b91b9508d13a756c80e180edcbc3161ac40a15b
  0x47f5a6397b79deaaca5c456503e7c1a49357b942
  0xd264fe39c90899434733b658e21886b8e39cc6a0
  0xe4bf16b70dd19404d842f057e7d9fc7337e52e43
  0x0ab2ff3a85eb341ff9f9307858b94904a42c5b45
  0x4334ff1dd9fa46667b718a6f8a489c3d0079b44b
  0x06c51f1fc98ff7cdee2b076ec5c8999af829b221
  0xf8905f26dff17d11859b89954a437bb85787d9a3
  0x9def3f176f98e0844170fa2336f01cc9d5f5e422
